# Simple Neural Network from Scratch

Simple program wich implements a Neural Network with customizable number of input features and output classes.

Example:

```python
nn = NeuralNetwork(n_input_features = 3, output_tpl = (1, sigmoid), hidden_layers = [(2, sigmoid)])

training_inputs = np.array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_outputs = np.array([[0, 1, 1, 0]]).T

nn.train(training_inputs, training_outputs, epochs = 20000, learning_rate = 0.1)
```

Main variables involved:

* $m$ number of samples used for the training process
* $L$ total number of layers (hidden + input values + output layer)

In every layer we then have the following variables:

* $n$ input features (i.e. number of neurons in layer 0)
* $N$ number of neurons


## Training 

The training is made of two main processes, which are repeated a number of times equal to `epochs` in the example before.

### Forward Propagation


### Backward Propagation



In [401]:
import numpy as np

In [393]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

def sigmoid_gradient(z):
    return(sigmoid(z)*(1-sigmoid(z)))

In [394]:
def get_cost_value(y_best, y_true):
    cost = - np.mean(np.dot(np.log(y_best).T, y_true) + np.dot(np.log(1 - y_best).T, 1 - y_true))
    return cost

In [395]:
class Layer:
    '''
    Every input (x) is a matrix (m x n) with m->n_samples and n->n_features 
    Given N->n_neurons, the weight matrix (W) is defined as (n x N), so that:
    dim(x dot W) = dim(z)
    (m x n) x (n x N) = (m x N)
    '''
    def __init__(self, *args):
        self.W = None
        self.b = None
        self.activation_function = None
        if args:
            n_features, n_neurons, activation_function = args
            self.W = np.random.rand(n_features, n_neurons)
            self.b = np.zeros((1, n_neurons))
            self.activation_function = activation_function
            
    def forward(self, A_prev):
        self.Z = np.dot(A_prev, self.W) + self.b
        self.A = self.activation_function(self.Z)
        
    def backward(self, delta, Z_prev, A_prev):
        self.delta = delta
        self.nabla_W = np.dot(A_prev.T, self.delta)
        self.nabla_b = np.sum(self.delta)
        self.delta_prev = np.dot(self.delta, self.W.T) * sigmoid_gradient(Z_prev)
        
    def update(self, learning_rate):
        self.W -= learning_rate * self.nabla_W
        self.b -= learning_rate * self.nabla_b

In [396]:
class NeuralNetwork:
    def __init__(self, n_input_features, output_tpl, hidden_layers = None):
        np.random.seed(0)
        if hidden_layers is None:
            hidden_layers = []
        layers_tpl = hidden_layers + [output_tpl]
        self.layers = []
        layers_inputs = [n_input_features] + [tpl[0] for tpl in layers_tpl[:-1]]
        for n_features, n_neurons, activation_function in zip(layers_inputs, 
                                                             [tpl[0] for tpl in layers_tpl], 
                                                             [tpl[1] for tpl in layers_tpl]):
            self.layers.append(Layer(n_features, n_neurons, activation_function))
        self.layers.insert(0, Layer()) # Layer 0, it has only self.a
        
    def forward(self, inputs):
        self.layers[0].A = inputs
        self.layers[0].Z = inputs
        for index, layer in enumerate(self.layers[1:]):
            layer.forward(self.layers[index].A)
        self.y_best = self.layers[-1].A
            
    def backward(self, y_true, learning_rate):
        delta = (self.layers[-1].A - y_true) * sigmoid_gradient(self.layers[-1].Z) 
        Z_prev = self.layers[-2].Z
        A_prev = self.layers[-2].A
        for index, layer in reversed(list(enumerate(self.layers[1:]))):
            layer.backward(delta, Z_prev, A_prev)
            delta = layer.delta_prev
            Z_prev = self.layers[index - 1].Z
            A_prev = self.layers[index - 1].A
            layer.update(learning_rate)
        
    def train(self, inputs, y_true, epochs, learning_rate):
        self.cost_history = []
        for epoch in range(epochs):
            self.forward(inputs)
            cost = get_cost_value(self.y_best, y_true)
            self.cost_history.append(cost)
            self.backward(y_true, learning_rate)
            if epoch%100 == 0:
                print('Epoch: {}, Loss: {}'.format(epoch, cost))

In [397]:
# API
#nn = NeuralNetwork(n_input_features = 3, output_tpl = (1, sigmoid), hidden_layers = [(2, sigmoid), (2, sigmoid)])
nn = NeuralNetwork(n_input_features = 3, output_tpl = (1, sigmoid), hidden_layers = [(2, sigmoid)])
#nn = NeuralNetwork(n_input_features = 3, output_tpl = (1, sigmoid))

In [398]:
training_inputs = np.array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_outputs = np.array([[0, 1, 1, 0]]).T

In [399]:
nn.train(training_inputs, training_outputs, 20000, 0.1)

Epoch: 0, Loss: 3.1069754330760464
Epoch: 100, Loss: 2.591495173097795
Epoch: 200, Loss: 2.3705301395693468
Epoch: 300, Loss: 1.975173336162457
Epoch: 400, Loss: 1.5267753534898993
Epoch: 500, Loss: 1.170459612462161
Epoch: 600, Loss: 0.9282097506472109
Epoch: 700, Loss: 0.7663134837899209
Epoch: 800, Loss: 0.6543249120286788
Epoch: 900, Loss: 0.5733931951701743
Epoch: 1000, Loss: 0.5125131995199665
Epoch: 1100, Loss: 0.46514028210438224
Epoch: 1200, Loss: 0.4272315131837394
Epoch: 1300, Loss: 0.3961864484405987
Epoch: 1400, Loss: 0.3702684569693821
Epoch: 1500, Loss: 0.34827835164300447
Epoch: 1600, Loss: 0.3293634165802995
Epoch: 1700, Loss: 0.31290155255798835
Epoch: 1800, Loss: 0.2984285973706833
Epoch: 1900, Loss: 0.285591322012677
Epoch: 2000, Loss: 0.2741161872385674
Epoch: 2100, Loss: 0.2637880582599279
Epoch: 2200, Loss: 0.25443537904142066
Epoch: 2300, Loss: 0.2459196378380391
Epoch: 2400, Loss: 0.23812774578371895
Epoch: 2500, Loss: 0.23096643219122576
Epoch: 2600, Loss: 0.2

In [400]:
print(nn.y_best)

[[0.01795721]
 [0.9863606 ]
 [0.98685253]
 [0.01748453]]
